In [1]:
import json
import numpy as np
import pandas as pd
from nltk.stem.snowball import FrenchStemmer
import unicodedata
import re
import math

In [11]:
df=pd.read_csv("Data/locations/commune2021.csv")
communes_ = np.array(df["LIBELLE"])
_insee_communes, counts = np.unique(communes_, return_counts=True)
insee_communes = np.unique([x.lower() for x in _insee_communes])

In [12]:
_dbPedia = json.load(open("Data/dbPedia_extracted_demonyms3.json"))
dbPedia = { x.lower():[v_.lower() for v_ in v] for x,v in _dbPedia.items()}
_infobox = json.load(open("Data/infobox_extracted_demonyms3.json"))
infobox = { x.lower():[v_.lower() for v_ in v] for x,v in _infobox.items()}
habitants_com =  json.load(open("Data/habitants_com_extracted_demonyms4.json"))
_infobox_wikidata = json.load(open("Data/wikidata_infobox_extracted_demonyms_final.json"))
infobox_wikidata = { x.lower():[v_.lower() for v_ in v] for x,v in _infobox_wikidata.items()}

In [16]:
merged = {}
for c in insee_communes:
    if c in infobox_wikidata or c in habitants_com:
        s1 = set([x.lower() for x in infobox_wikidata.get(c, [])])
        s2 = set([x.lower() for x in habitants_com.get(c, [])])
        s = s1.union(s2)
        if len(s)>0:
            merged[c] = list(s)
    elif c in dbPedia:
        merged[c] = [x.lower() for x in dbPedia[c]]
    

In [15]:
x = 0
for c in insee_communes:
    if c in dbPedia and c not in infobox_wikidata and c not in habitants_com:
        print(c)
        x+=1
x

aime-la-plagne
amfreville-saint-amand
ampriani
ancenis-saint-géréon
ancy-dornot
argences en aubrac
argentat-sur-dordogne
arvière-en-valromey
aspach-michelbach
athis-val de rouvre
aunac-sur-charente
auneau-bleury-saint-symphorien
aurseulles
autrans-méaudre en vercors
aÿ-champagne
bagnoles de l'orne normandie
balleroy-sur-drôme
ballon-saint mars
bassillac et auberoche
baugé-en-anjou
bazouges cré sur loir
beaufort-en-anjou
beaumontois en périgord
beaussais-vitré
belforêt-en-perche
bellevigne
bellevigne-en-layon
bellevigne-les-châteaux
belsentes
bernay-neuvy-en-champagne
blaison-saint-sulpice
blancs-coteaux
bon repos sur blavet
bordes-uchentein
bors (canton de tude-et-lavalette)
bosroumois
boulazac isle manoire
bourgs sur colagne
brantôme en périgord
bray-saint-aignan
brissac loire aubance
brunstatt-didenheim
bure
bâgé-dommartin
bédouès-cocurès
campana
campi
canavaggia
capavenir vosges
carentan-les-marais
carnoy-mametz
casevecchie
castellare-di-mercurio
castelnau montratier-sainte alauzie


304

In [17]:
len(merged)

30422

In [19]:
with open("Data/final/gentiles_merged.json", "w") as fp:
        json.dump(merged,fp)

In [2]:
def stem_nltk(word):
    stemmer = FrenchStemmer()
    return stemmer.stem(word)

In [23]:
all = json.load(open("Data/final/gentiles_merged.json"))
reversedd , reversed_stemmed = {}, {}

In [24]:
for commune, demonyms in all.items():
    for demonym in demonyms:
        if demonym in reversedd:
            reversedd[demonym].add(commune)
        else:
            reversedd[demonym] = set([commune])

        stemmed = stem_nltk(demonym)
        if stemmed in reversed_stemmed:
            reversed_stemmed[stemmed].add(commune)
        else:
            reversed_stemmed[stemmed] = set([commune])
            
for k,v in reversedd.items():
    reversedd[k] = list(v)
for k,v in reversed_stemmed.items():
    reversed_stemmed[k] = list(v)

with open("Data/final/gentiles_merged_reversed.json", "w") as fp:
         json.dump(reversedd,fp)
with open("Data/final/gentiles_merged_reversed_stemmed.json", "w") as fp:
         json.dump(reversed_stemmed,fp)

In [25]:
def get_location_demonym_dict(demonym, stemmed_dict):
    stemmed = stem_nltk(demonym)
    if stemmed in stemmed_dict:
        return stemmed_dict[stemmed]
    return -1

In [26]:
get_location_demonym_dict("bellifontaines", reversed_stemmed)

['belfonds', 'bellefontaine', 'fontainebleau', 'beaulieu-les-fontaines']

In [27]:
print(len(reversedd), len(reversed_stemmed))

56696 35262


In [3]:
all = json.load(open("Data/regions_demonyms.json"))
reversed_stemmed = {}

for dep, demonyms in all.items():
    for demonym in demonyms:
        stemmed = stem_nltk(demonym)
        if stemmed in reversed_stemmed:
            reversed_stemmed[stemmed].add(dep.lower())
        else:
            reversed_stemmed[stemmed] = set([dep.lower()])

for k,v in reversed_stemmed.items():
    reversed_stemmed[k] = list(v)

with open("Data/regs_stemmed_reversed.json", "w") as fp:
         json.dump(reversed_stemmed,fp)


In [30]:
infobox_wikidata["val de virvée"]

['virvéens']

In [32]:
all

{'Ain': ['Aindinois', 'Aindinoises'],
 'Aisne': ['Axonais', 'Axonaises'],
 'Allier': ['Bourbonnais', 'Bourbonnaises'],
 'Alpes-de-Haute-Provence': ['Bas-Alpins', 'Bas-Alpines'],
 'Hautes-Alpes': ['Haut-Alpins', 'Haut-Alpines'],
 'Alpes-Maritimes': ['Maralpins', 'Maralpines'],
 'Ardèche': ['Ardéchois', 'Ardéchoises'],
 'Ardennes': ['Ardennais', 'Ardennaises'],
 'Ariège': ['Ariégeois', 'Ariégeoises'],
 'Aube': ['Aubois', 'Auboises'],
 'Aude': ['Audois', 'Audoises'],
 'Aveyron': ['Aveyronnais', 'Aveyronnaises'],
 'Bouches-du-Rhône': ['Buccorhodaniens', 'Buccorhodaniennes'],
 'Calvados': ['Calvadosiens', 'Calvadosiennes'],
 'Cantal': ['Cantaliens', 'Cantaliennes', 'Cantalous', 'Cantalouses'],
 'Charente': ['Charentais', 'Charentaises'],
 'Charente-Maritime': ['Charentais maritimes', 'Charentaises maritimes'],
 'Cher': ['Chériens', 'Chériennes', 'Berrichons', 'Berrichonnes'],
 'Corrèze': ['Corréziens', 'Corréziennes'],
 'Corse-du-Sud': ['Corses du Sud'],
 'Haute-Corse': ['Corses du Nord'],


In [33]:
len(all)

106